In [217]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import pearsonr
import re

nhl_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

big4 = 'NBA'

In [218]:
sports = ['Metropolitan area', 'Population', 'NFL','MLB','NBA','NHL']
cities.columns = sports 
def cleaning(item):
    #s = re.sub('\[.*\]', '', item)
    s = re.sub(r'([a-z])([A-Z])', r'\1 \2', item)
    return s

def cleaning_2(item):
    s = re.sub('\[.*\]', '', item)
    #s = re.sub(r'([a-z])([A-Z])', r'\1 \2', item)
    return s

def cleaning_3(item):
    item.strip()
    s = re.sub('\**\s\([0-9]*\)$', '', item)
    return s


def apply_cleaning(clen_fun, dataframe, columns):
    for fun in clen_fun:
        for col in columns:
            dataframe[col]  = dataframe[col].apply(fun)
    return dataframe

functions = [cleaning, cleaning_2, cleaning_3]

apply_cleaning(functions, cities, sports)
cities.sort_values('Metropolitan area', axis = 0, inplace=True)
cities.reset_index(drop=True,inplace=True)
cities.head()

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,Atlanta,5789700,Falcons,Braves,Hawks,
1,Baltimore,2798886,Ravens,Orioles,,—
2,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
3,Buffalo,1132804,Bills,,,Sabres
4,Calgary,1392609,—,—,—,Flames


In [219]:
city_replace = {'Dallas–Fort Worth':'Dallas','Miami–Fort Lauderdale':'Miami','Minneapolis–Saint Paul':'Minneapolis','New York City':'New York','San Francisco Bay Area':'San Francisco','Tampa Bay Area':'Tampa Bay','Washington, D.C.':'Washington','':'None','—':'None','— ':'None'}
cities.replace(to_replace=city_replace,value = None, inplace=True)
cities

,Metropolitan area,Population,NFL,MLB,NBA,NHL
0,Atlanta,5789700,Falcons,Braves,Hawks,None
1,Baltimore,2798886,Ravens,Orioles,None,None
2,Boston,4794447,Patriots,Red Sox,Celtics,Bruins
3,Buffalo,1132804,Bills,None,None,Sabres
4,Calgary,1392609,None,None,None,Flames
5,Charlotte,2474314,Panthers,None,Hornets,None
6,Chicago,9512999,Bears,Cubs White Sox,Bulls,Blackhawks
7,Cincinnati,2165139,Bengals,Reds,None,None
8,Cleveland,2055612,Browns,Indians,Cavaliers,None
9,Columbus,2041520,None,None,None,Blue Jackets


In [220]:
nhl_df = nhl_df[nhl_df['year'] == 2018]
print(nhl_df.shape)
#nhl_df.drop([0,9,18,26], axis = 0, inplace= True)
nhl_df = nhl_df[['team','W','L',]]
replace_nhl = {'Brooklyn':'New York','Minnesota':'Minneapolis','Golden State':'San Francisco','Indiana':'Indianapolis','Portland Trail':'Portland','Blazers':'Trail Blazers','Utah':'Salt Lake City'}

(30, 10)


In [221]:
apply_cleaning(functions, nhl_df, list(nhl_df.columns))
nhl_df

,team,W,L
0,Toronto Raptors,59,23
1,Boston Celtics,55,27
2,Philadelphia 76ers,52,30
3,Cleveland Cavaliers,50,32
4,Indiana Pacers,48,34
5,Miami Heat,44,38
6,Milwaukee Bucks,44,38
7,Washington Wizards,43,39
8,Detroit Pistons,39,43
9,Charlotte Hornets,36,46


In [222]:
def splitname(row):
    row['Team']=row['team'].split(" ")[-1]
    s=row['team'].split(" ")[:-1]
    row['City']=' '.join(map(str, s)) #make it a single string instead of a list
    return row

nhl_df=nhl_df.apply(splitname, axis='columns')
del(nhl_df['team'])
nhl_df.replace(to_replace=replace_nhl,value=None,inplace=True)
nhl_df = nhl_df.set_index('City')
nhl_df.sort_values('City', axis=0, inplace=True)
nhl_df

,W,L,Team
City,,,
Atlanta,24,58,Hawks
Boston,55,27,Celtics
Charlotte,36,46,Hornets
Chicago,27,55,Bulls
Cleveland,50,32,Cavaliers
Dallas,24,58,Mavericks
Denver,46,36,Nuggets
Detroit,39,43,Pistons
Houston,65,17,Rockets


In [223]:
cities_nhl = cities[cities[big4] != 'None'] #take all the cities that have a "NBA" team
cities_nhl = cities_nhl[['Metropolitan area','Population',big4]] # filter only interesting parameters
cities_nhl = cities_nhl.set_index('Metropolitan area') 
cities_nhl.index.rename('City', inplace=True) 
print('nr of cities: {}, nr of {} teams: {}'.format(cities_nhl.shape[0],big4,nhl_df.shape[0]))
cities_nhl

nr of cities: 28, nr of NBA teams: 30


,Population,NBA
City,,
Atlanta,5789700,Hawks
Boston,4794447,Celtics
Charlotte,2474314,Hornets
Chicago,9512999,Bulls
Cleveland,2055612,Cavaliers
Dallas,7233323,Mavericks
Denver,2853077,Nuggets
Detroit,4297617,Pistons
Houston,6772470,Rockets


In [224]:
#cities_nhl[~cities_nhl.index.isin(nhl_df.index)] #show cities that are not in the NBA dataframe
nhl_df[~nhl_df.index.isin(cities_nhl.index)] #Check what cities are not matching the names

,W,L,Team
City,,,


In [225]:
cities_nhl = cities_nhl.merge(nhl_df, left_index=True, right_index=True)
cities_nhl.drop(labels=big4, axis = 1, inplace=True)
cities_nhl['W/L Ratio'] = cities_nhl['W'].astype('float64')/(cities_nhl['L'].astype('float64') + cities_nhl['W'].astype('float64'))
print(cities_nhl.shape)
cities_nhl


(30, 5)


,Population,W,L,Team,W/L Ratio
City,,,,,
Atlanta,5789700,24,58,Hawks,0.292683
Boston,4794447,55,27,Celtics,0.670732
Charlotte,2474314,36,46,Hornets,0.439024
Chicago,9512999,27,55,Bulls,0.329268
Cleveland,2055612,50,32,Cavaliers,0.609756
Dallas,7233323,24,58,Mavericks,0.292683
Denver,2853077,46,36,Nuggets,0.560976
Detroit,4297617,39,43,Pistons,0.475610
Houston,6772470,65,17,Rockets,0.792683


In [226]:
cities_nhl['Avg. Ratio'] = cities_nhl.groupby(cities_nhl.index).agg(np.mean)
cities_nhl['Population'] = cities_nhl['Population'].astype('int64')
corr1 = cities_nhl['Population'].corr(cities_nhl['Avg. Ratio'],method='pearson')
print('answer to question 1: {}'.format(corr1))

answer to question 1: -0.25421734896460507


In [227]:
ans_df = cities_nhl[['Population','Avg. Ratio']]
ans_df.drop_duplicates(inplace=True)

In [228]:
population_by_region = list(ans_df['Population'])
win_loss_by_region = list(ans_df['Avg. Ratio'])

In [229]:
stats.pearsonr(population_by_region, win_loss_by_region)[0]

-0.17657160252844623

In [230]:
population_by_region

[5789700,
 4794447,
 2474314,
 9512999,
 2055612,
 7233323,
 2853077,
 4297617,
 6772470,
 2004230,
 13310447,
 1342842,
 6066387,
 1572482,
 3551036,
 1268883,
 20153634,
 1373211,
 2441257,
 6070500,
 4661537,
 2424955,
 2296418,
 1186187,
 2429609,
 6657982,
 5928040,
 6131977]

In [232]:
len(win_loss_by_region)

28